In [0]:
import pandas as pd
import numpy as np
import os

from imageio import imread
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline
from itertools import chain
import torch
import torchvision
import itertools
import logging
import warnings
warnings.filterwarnings('ignore')

from torch.nn import Sigmoid
from torch import Tensor, LongTensor
from torch.autograd import Variable
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from PIL import Image

logging.basicConfig(format='%(message)s',level = logging.INFO)

In [0]:
fileList = os.listdir("D:\\2")

In [0]:
df['ind'] = df.filename.apply(lambda x : 1 if "/2/" in x else 0)

In [33]:
subDf = df[df.ind==1]
subDf.head()


,DummyID,filename,train,imageName,label,BMI,ind
0,76093408,FullRes/2/76093408_L_CC_1.jpg,True,76093408_L_CC_1.jpg,1,31.8,1
1,76071433,FullRes/2/76071433_L_CC_1.jpg,False,76071433_L_CC_1.jpg,1,24.3,1
4,76863896,FullRes/2/76863896_R_CC_1.jpg,False,76863896_R_CC_1.jpg,0,23.4,1
5,76428052,FullRes/2/76428052_L_MLO_1.jpg,False,76428052_L_MLO_1.jpg,0,32.9,1
11,5170186,FullRes/2/5170186_R_MLO_1.jpg,False,5170186_R_MLO_1.jpg,1,27.2,1


In [0]:
dropList = []

for i in range(len(subDf)):

  filename= subDf.iloc[i].filename.split("/")[-1]

  if filename not in fileList:
    dropList.append(i)
    

In [35]:
subDf = subDf.reset_index()
subDf.drop('index',inplace =True, axis = 1)
subDf.head()

,DummyID,filename,train,imageName,label,BMI,ind
0,76093408,FullRes/2/76093408_L_CC_1.jpg,True,76093408_L_CC_1.jpg,1,31.8,1
1,76071433,FullRes/2/76071433_L_CC_1.jpg,False,76071433_L_CC_1.jpg,1,24.3,1
2,76863896,FullRes/2/76863896_R_CC_1.jpg,False,76863896_R_CC_1.jpg,0,23.4,1
3,76428052,FullRes/2/76428052_L_MLO_1.jpg,False,76428052_L_MLO_1.jpg,0,32.9,1
4,5170186,FullRes/2/5170186_R_MLO_1.jpg,False,5170186_R_MLO_1.jpg,1,27.2,1


In [0]:
subDf.drop(dropList,axis=0,inplace = True)

In [0]:
cleanDf = df[df.ind==0]


finalDf = cleanDf.append(subDf)

In [40]:
len(finalDf)

14720

In [0]:
finalDf.to_csv("D:\\LabelFile_2000_updated.csv",index = False, index_label=False)

In [0]:
df = pd.read_csv('D:\\LabelFile_2000.txt')

In [8]:
df.head()

,DummyID,filename,train,imageName,label,BMI
0,76093408,FullRes/2/76093408_L_CC_1.jpg,True,76093408_L_CC_1.jpg,1,31.8
1,76071433,FullRes/2/76071433_L_CC_1.jpg,False,76071433_L_CC_1.jpg,1,24.3
2,75828260,PennExtra_3500/75828260_FOR-PROCESSING_L_MLO_1...,False,75828260_FOR-PROCESSING_L_MLO_1.jpg,1,25.7
3,76661032,FullRes/3/76661032_L_MLO_1.jpg,False,76661032_L_MLO_1.jpg,1,24.4
4,76863896,FullRes/2/76863896_R_CC_1.jpg,False,76863896_R_CC_1.jpg,0,23.4


In [0]:
trainData = df[df.train == False]
validData = df[df.train == True]

In [0]:
dataFolder = "D:\\"

In [0]:
class CancerDataset(torch.utils.data.Dataset):
  def __init__(self, dataFolder, dataList):
    self.data = dataList
    
  def __len__(self):
    return len(self.data)
    
  def __getitem__(self, index):
    filename = self.data.iloc[index].filename
    if 'FullRes' in filename :
        #Removes the starting "FullRes" from the filename
        filename = filename[7:]
    data = imread(os.path.join(dataFolder, filename))
    data = Image.fromarray(data)
    data = data.resize((512,512))
    data = np.array(data)
    data = data/256.0 
    data = torchvision.transforms.ToTensor()(data)
    data = data.float()
    label = self.data.iloc[index].label
    bmi = self.data.iloc[index].BMI
    return data, np.array([label, bmi]).type(float)

In [0]:
trainDataset = Dataset(dataFolder, trainData) 
validDataset = Dataset(dataFolder, validData) 

In [16]:
len(trainDataset)

12984